# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [ ]:
!pip install kaggle

In [ ]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

In [ ]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

In [ ]:
train_df.describe()

In [ ]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

In [ ]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

In [ ]:
linear_reg.coef_

In [ ]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

In [ ]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

In [ ]:
log_reg.predict(test_case)[0]

In [ ]:
help(log_reg.predict)

In [ ]:
log_reg.predict_proba(test_case)[0]

In [ ]:
# What's the math?
log_reg.coef_

In [ ]:
log_reg.intercept_

In [ ]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [ ]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [ ]:
# Live - let's try absenteeism!

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [ ]:
'''
This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.
'''

In [ ]:
# !scp /c/users/jhump/Downloads/fma_metadata.zip -- does not return zip file

In [ ]:
# !unzip fma_metadata.zip -- not needed with local workflow

In [1]:
import pandas as pd
tracks = pd.read_csv('C:\\Users\\jhump\\Desktop\\fma_metadata\\tracks.csv')

C:\Users\jhump\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
tracks.describe()

In [2]:
pd.set_option('display.max_columns', None)  # Unlimited columns
# tracks.head()

In [3]:
tracks.shape

(106576, 53)

In [34]:
import numpy as np

tracks_copy = tracks.copy()
new_header = tracks_copy.iloc[0]
tracks_copy = tracks_copy[1:]
tracks_copy.columns = new_header
tracks_copy = tracks_copy.rename(columns={np.nan: 'track_id'})
tracks_copy = tracks_copy.drop(index=[1])
tracks_copy.head()
# tracks_copy.shape

,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
5,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,[],Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3,2008-11-26 01:42:55,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"['philly', 'kurt vile']",http://kurtvile.com,NaN,training,small,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
6,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,[],Niris,13,Album,1990-01-01 00:00:00,2011-01-01 00:00:00,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2,2008-11-26 01:42:52,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"['instrumentals', 'experimental pop', 'post pu...",NaN,NaN,training,large,256000,0,NaN,2008-11-26 01:48:56,2008-01-01 00:00:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [5]:
tracks_copy.isna().sum()

0
track_id                  0
comments                  0
date_created           3529
date_released         36280
engineer              91279
favorites                 0
id                        0
information           23425
listens                   0
producer              88514
tags                      0
title                  1025
tracks                    0
type                   6508
active_year_begin     83863
active_year_end      101199
associated_labels     92303
bio                   35418
comments                  0
date_created            856
favorites                 0
id                        0
latitude              62030
location              36364
longitude             62030
members               59725
name                      0
related_projects      93422
tags                      0
website               27318
wikipedia_page       100993
split                     0
subset                    0
bit_rate                  0
comments                  0
composer          

In [20]:
# let's drop all columns with NaN's equal to 90% of total rows
na_cutoff = int(tracks_copy.shape[0] * 0.9)

for num in tracks_copy.isna().sum():
    if num >= na_cutoff:
        print(num)

# by quickly scanning the resultant isna numbers above, can see that...
# 101199 is for active_year_end
# 100993, wikipedia_page
# 102904, composer
# 100415, date_recorded
# 104225, information
# 106263, lyricist
# 105311, publisher

# will drop each of these columns from tracks_copy
# tracks_copy = tracks_copy.drop(['active_year_end', 'wikipedia_page', 'composer',
#                                'date_recorded', 'information', 'lyricist', 'publisher'], axis=1)


In [21]:
print('new number of columns in tracks_copy is:', tracks_copy.shape[1])

new number of columns in tracks_copy is: 45


In [18]:
tracks_copy.columns
tracks_copy_cols_map = zip([_ for _ in range(46)], tracks_copy.columns)
tracks_copy_cols_dict = dict(tracks_copy_cols_map)
tracks_copy_cols_dict

{0: 'track_id',
 1: 'comments',
 2: 'date_created',
 3: 'date_released',
 4: 'engineer',
 5: 'favorites',
 6: 'id',
 7: 'listens',
 8: 'producer',
 9: 'tags',
 10: 'title',
 11: 'tracks',
 12: 'type',
 13: 'active_year_begin',
 14: 'associated_labels',
 15: 'bio',
 16: 'comments',
 17: 'date_created',
 18: 'favorites',
 19: 'id',
 20: 'latitude',
 21: 'location',
 22: 'longitude',
 23: 'members',
 24: 'name',
 25: 'related_projects',
 26: 'tags',
 27: 'website',
 28: 'split',
 29: 'subset',
 30: 'bit_rate',
 31: 'comments',
 32: 'date_created',
 33: 'duration',
 34: 'favorites',
 35: 'genre_top',
 36: 'genres',
 37: 'genres_all',
 38: 'interest',
 39: 'language_code',
 40: 'license',
 41: 'listens',
 42: 'number',
 43: 'tags',
 44: 'title'}

In [ ]:
# STRETCH GOAL
# rearrange order of tracks_copy columns for more logical, informative appearance
# tracks_copy = tracks_copy.iloc[0, 11, 2:11, 1, 12:52]
# cols = tracks_copy.columns.tolist()
# cols

In [37]:
tracks_copy.genre_top.value_counts().sum()
# tracks_copy.producer.unique()

49598

In [39]:
# check for 'genre_top' whether value_counts().sum() + isna().sum() == tracks_copy.shape[0]
tracks_copy.shape[0] == tracks_copy.genre_top.value_counts().sum() + tracks_copy.genre_top.isna().sum()

True

In [76]:
# create new tracks_copy_1 df that has no NaN's in genre_top
tracks_copy_1 = tracks_copy.copy()
tracks_copy_1 = tracks_copy_1[tracks_copy_1.genre_top.notna()]
tracks_copy_1.shape

(49598, 53)

In [77]:
tracks_copy_1.isna().sum()

0
track_id                 0
comments                 0
date_created          1051
date_released        16086
engineer             40687
favorites                0
id                       0
information          10154
listens                  0
producer             39372
tags                     0
title                  309
tracks                   0
type                  2047
active_year_begin    40016
active_year_end      47200
associated_labels    43455
bio                  16194
comments                 0
date_created           215
favorites                0
id                       0
latitude             28618
location             17488
longitude            28618
members              27653
name                     0
related_projects     44233
tags                     0
website              12693
wikipedia_page       46864
split                    0
subset                   0
bit_rate                 0
comments                 0
composer             48245
date_created             0

In [ ]:
# STRETCH GOALS
# mylist = list(tracks_copy.select_dtypes(include=['int64', 'float64']).columns)
# mylist
# tracks_copy.dtypes
# tracks_copy.date_recorded.unique()
# tracks_copy['date_recorded'] = pd.to_numeric(list(tracks_copy['date_recorded']), errors='coerce')
# tracks_copy['date_recorded'].unique()

In [ ]:
# STRETCH GOALS
# import datetime as dt
# # convert timestamp strings into MMDDHH integer
# # first, write function to do conversion

# def convert_timestamp_to_int(val):
#     """
#     https://stackoverflow.com/questions/54046862/python-convert-timestamp-string-to-mmddhh-integer
#     """
#     date_string = dt.datetime.strptime(val, '%Y-%m-%d %H:%M:%S')
#     different_date_string = dt.datetime.strftime(date_string, '%m%d%H')
#     return int(different_date_string)

# # second, apply function to all values in date_recorded column
# tracks_copy['date_recorded'] = tracks_copy['date_recorded'].apply(convert_timestamp_to_int)

In [78]:
tracks_copy_1.interest.isna().sum()

0

In [79]:
# build Logistic Regression model to predict "genre_top" for any given album
# one way to do this involves converting string column names to numeric names - was unable to do this in efficient time
# another preprocessing route that is faster is to go with columns that are already numeric

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts

# would've liked to have used 'language_code', but tracks_copy.language_code.isna().sum() = 91550
# tracks_copy.duration = tracks_copy.duration.astype(float)

# tracks_copy = tracks_copy[tracks_copy['genre_top'].notnull()] -- unsure whether to use this or not
# genre_top_dict = dict(zip(sorted(tracks_copy.genre_top.unique()), (_ for _ in range(16))))

# preprocess and generate data
X = tracks_copy_1[['tracks', 'duration', 'interest']]
tracks_copy_1.genre_top = tracks_copy_1.genre_top.replace(to_replace={'Blues': 0, 'Classical': 1, 'Country': 2,
                                                                  'Easy Listening': 3, 'Electronic': 4,
                                                                  'Experimental': 5, 'Folk': 6, 'Hip-Hop': 7,
                                                                  'Instrumental': 8, 'International': 9,
                                                                  'Jazz': 10, 'Old-Time / Historic': 11,
                                                                  'Pop': 12, 'Rock': 13, 'Soul-RnB': 14,
                                                                  'Spoken': 15})
y = tracks_copy_1.genre_top
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.50, random_state=42)

'''
ValueError: could not convert string to float: "['ballad', 'epic', 'rockabilly', 'curse', 'hex', 'hard rock', 'cauldron', 'witches', 'creepy', 'black cats']"
'''

'\nValueError: could not convert string to float: "[\'ballad\', \'epic\', \'rockabilly\', \'curse\', \'hex\', \'hard rock\', \'cauldron\', \'witches\', \'creepy\', \'black cats\']"\n'

In [80]:
y.value_counts().sum() == tracks_copy_1.shape[0]

True

In [81]:
log_reg_train = LogisticRegression().fit(X_train, y_train)
print('score for training data:', log_reg_train.score(X_train, y_train))

log_reg_test = LogisticRegression().fit(X_test, y_test)
print('score for testing data:', log_reg_test.score(X_test, y_test))

score for training data: 0.37170853663454173
score for testing data: 0.3654179604016291


In [82]:
# Now, look at the simple majority classifier for y_test **
y_test.value_counts(normalize=True)

# Comment: my initial LogReg model is more highly predictive than the SMC, but only by about 6%

13    0.287834
5     0.211742
4     0.188758
7     0.072705
6     0.056212
12    0.046655
8     0.042582
9     0.027945
1     0.023751
10    0.011291
11    0.011250
15    0.008468
2     0.004315
14    0.003670
0     0.002298
3     0.000524
Name: genre_top, dtype: float64

In [86]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X_test, y_test)
log_reg.score(X_test, y_test)

# log_reg.predict(y_test[[]])
print('log_reg coefficient is:', log_reg.coef_)
print('log_reg intercept is:', log_reg.intercept_)

def sigmoid(x):
  return 1 / (1 + np.e**(-x))

# need to create test_case
# sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

log_reg coefficient is: [[-3.97659376e-02 -1.72029565e-03  2.53674427e-06]
 [ 1.23805434e-03  5.05833308e-04  7.90626808e-06]
 [-9.40177710e-02 -3.07733638e-03  2.41059718e-07]
 [-1.43878835e-01 -1.94993513e-03  2.10031519e-07]
 [-3.48612089e-03 -1.72700326e-04  4.56802303e-06]
 [ 6.12239251e-03  1.58948154e-03 -3.25064443e-04]
 [-6.59762600e-03 -8.77522487e-04 -6.49262160e-05]
 [-6.32510834e-03 -2.35068027e-03  8.84367698e-07]
 [-2.38473414e-04 -2.44847466e-04  5.29374064e-06]
 [-1.37311801e-01 -2.04998314e-03  6.71665940e-07]
 [-1.04963757e-01 -1.09072818e-04  1.20414719e-06]
 [-3.73256471e-02 -5.29091292e-03  1.38631979e-06]
 [ 5.68507465e-03 -3.02007902e-03 -6.58359950e-06]
 [-2.42633395e-02 -1.40430798e-03 -1.13870061e-04]
 [-1.27289382e-01 -5.13965167e-03  5.92774647e-07]
 [ 1.89739912e-04  1.21629094e-03 -8.60215906e-05]]
log_reg intercept is: [-5.08767124e+00 -3.91580590e+00 -3.66188619e+00 -5.39240202e+00
 -1.35652010e+00 -1.44222193e+00 -2.36677369e+00 -1.91341786e+00
 -3.117

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.